In [ ]:
import os
import shutil
import xml.etree.ElementTree as ET
from collections import defaultdict
import logging

# Setup logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

# Input and output folders
INPUT_DIR = r"C:\Users\krish\OneDrive\Desktop\Hackathon\src\Problem_3\train\original_dataset_from_kaggle"
OUTPUT_DIR = r"C:\Users\krish\OneDrive\Desktop\Hackathon\src\Problem_3\train\dataset"

# Target classes
TARGET_CLASSES = {"car", "motorbike", "bus"}

# Max samples per class
MAX_PER_CLASS = 800

# Create output directory
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Track counts
class_counts = defaultdict(int)

# Iterate over XML files
for file in os.listdir(INPUT_DIR):
    if not file.endswith(".xml"):
        continue

    xml_path = os.path.join(INPUT_DIR, file)
    base_name = os.path.splitext(file)[0]
    img_file = base_name + ".jpg"

    # Stop if all classes reached limit
    if all(class_counts[c] >= MAX_PER_CLASS for c in TARGET_CLASSES):
        logging.info("All classes reached 50 samples.")
        break

    try:
        tree = ET.parse(xml_path)
        root = tree.getroot()

        for obj in root.findall("object"):
            cls_name = obj.find("name").text.strip().lower()

            if cls_name not in TARGET_CLASSES:
                continue

            if class_counts[cls_name] >= MAX_PER_CLASS:
                continue

            # New filenames
            count = class_counts[cls_name] + 1
            out_img_name = f"{cls_name}{count}.jpg"
            out_xml_name = f"{cls_name}{count}.xml"

            out_img_path = os.path.join(OUTPUT_DIR, out_img_name)
            out_xml_path = os.path.join(OUTPUT_DIR, out_xml_name)

            # Copy files if image exists
            img_path = os.path.join(INPUT_DIR, img_file)
            if os.path.exists(img_path):
                shutil.copy(img_path, out_img_path)
                shutil.copy(xml_path, out_xml_path)
                class_counts[cls_name] += 1
                logging.info(f"Copied {img_file} -> {out_img_name} and {file} -> {out_xml_name}")
            else:
                logging.warning(f"Image {img_file} not found for {file}")

            # Stop early if all classes done
            if all(class_counts[c] >= MAX_PER_CLASS for c in TARGET_CLASSES):
                break

    except ET.ParseError:
        logging.error(f"Failed to parse XML: {file}")
        continue
    except Exception as e:
        logging.error(f"Error processing {file}: {str(e)}")
        continue

# Summary
logging.info("Dataset preparation complete.")
for cls in TARGET_CLASSES:
    logging.info(f"{cls}: {class_counts[cls]} samples")


2025-08-20 08:43:38,681 - INFO - Copied 01.jpg -> bus1.jpg and 01.xml -> bus1.xml
2025-08-20 08:43:38,688 - INFO - Copied 01.jpg -> bus2.jpg and 01.xml -> bus2.xml
2025-08-20 08:43:38,691 - INFO - Copied 01.jpg -> bus3.jpg and 01.xml -> bus3.xml
2025-08-20 08:43:38,694 - INFO - Copied 01.jpg -> motorbike1.jpg and 01.xml -> motorbike1.xml
2025-08-20 08:43:38,698 - INFO - Copied 01.jpg -> car1.jpg and 01.xml -> car1.xml
2025-08-20 08:43:38,702 - INFO - Copied 01.jpg -> car2.jpg and 01.xml -> car2.xml
2025-08-20 08:43:38,707 - INFO - Copied 01.jpg -> car3.jpg and 01.xml -> car3.xml
2025-08-20 08:43:38,712 - INFO - Copied 01.jpg -> car4.jpg and 01.xml -> car4.xml
2025-08-20 08:43:38,718 - INFO - Copied 02.jpg -> motorbike2.jpg and 02.xml -> motorbike2.xml
2025-08-20 08:43:38,722 - INFO - Copied 02.jpg -> motorbike3.jpg and 02.xml -> motorbike3.xml
2025-08-20 08:43:38,726 - INFO - Copied 02.jpg -> motorbike4.jpg and 02.xml -> motorbike4.xml
2025-08-20 08:43:38,730 - INFO - Copied 02.jpg -> 

In [22]:
import os
import xml.etree.ElementTree as ET
import matplotlib.pyplot as plt
import seaborn as sns
from collections import defaultdict
import numpy as np

# Input directory (adjust as needed)
INPUT_DIR = r"C:\Users\krish\OneDrive\Desktop\Hackathon\src\Problem_3\train\Dataset"

# Target classes
TARGET_CLASSES = {"car", "motorbike", "bus"}

# Initialize data structures
class_counts = defaultdict(int)
areas = defaultdict(list)
widths = defaultdict(list)
heights = defaultdict(list)

# Parse XML files
for file in os.listdir(INPUT_DIR):
    if file.endswith(".xml"):
        xml_path = os.path.join(INPUT_DIR, file)
        try:
            tree = ET.parse(xml_path)
            root = tree.getroot()
            for obj in root.findall("object"):
                cls_name = obj.find("name").text.strip().lower()
                if cls_name in TARGET_CLASSES:
                    class_counts[cls_name] += 1
                    bndbox = obj.find("bndbox")
                    xmin = float(bndbox.find("xmin").text)
                    ymin = float(bndbox.find("ymin").text)
                    xmax = float(bndbox.find("xmax").text)
                    ymax = float(bndbox.find("ymax").text)
                    width = xmax - xmin
                    height = ymax - ymin
                    area = width * height
                    areas[cls_name].append(area)
                    widths[cls_name].append(width)
                    heights[cls_name].append(height)
        except ET.ParseError:
            print(f"Error parsing {file}")
            continue

# Plot 1: Number of objects per class
plt.figure(figsize=(8, 6))
sns.barplot(x=list(class_counts.keys()), y=list(class_counts.values()))
plt.title("Number of Objects per Class")
plt.xlabel("Class")
plt.ylabel("Count")
plt.savefig("class_distribution.png")
plt.close()

# Plot 2: Box plot of bounding box areas
plt.figure(figsize=(8, 6))
data = [areas[cls] for cls in TARGET_CLASSES]
plt.boxplot(data, labels=TARGET_CLASSES)
plt.title("Bounding Box Area Distribution per Class")
plt.xlabel("Class")
plt.ylabel("Area (pixels²)")
plt.savefig("area_distribution.png")
plt.close()

# Plot 3: Box plot of bounding box widths
plt.figure(figsize=(8, 6))
data = [widths[cls] for cls in TARGET_CLASSES]
plt.boxplot(data, labels=TARGET_CLASSES)
plt.title("Bounding Box Width Distribution per Class")
plt.xlabel("Class")
plt.ylabel("Width (pixels)")
plt.savefig("width_distribution.png")
plt.close()

# Plot 4: Box plot of bounding box heights
plt.figure(figsize=(8, 6))
data = [heights[cls] for cls in TARGET_CLASSES]
plt.boxplot(data, labels=TARGET_CLASSES)
plt.title("Bounding Box Height Distribution per Class")
plt.xlabel("Class")
plt.ylabel("Height (pixels)")
plt.savefig("height_distribution.png")
plt.close()

print("Visualizations saved: class_distribution.png, area_distribution.png, width_distribution.png, height_distribution.png")

Visualizations saved: class_distribution.png, area_distribution.png, width_distribution.png, height_distribution.png


In [ ]:
import os
from ultralytics import YOLO
import matplotlib.pyplot as plt
import numpy as np

# Define paths
model_path = r"C:\Users\krish\OneDrive\Desktop\Hackathon\src\Problem_3\train\model.pt"
test_images_dir = r"C:\Users\krish\OneDrive\Desktop\Hackathon\src\Problem_3\test\test_data"
output_dir = r"C:\Users\krish\OneDrive\Desktop\Hackathon\src\Problem_3\inference_results"


# Create output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Load the trained model
model = YOLO(model_path)

# Perform inference on test images and save results
results = model.predict(source=test_images_dir, save=True, project=output_dir, name="Inference_Results", exist_ok=True)

print(f"Inference Result: {output_dir}/Inference_Results/")


image 1/20 C:\Users\krish\OneDrive\Desktop\Hackathon\src\Problem_3\test\Test_Data\data1.jpeg: 352x416 5 cars, 3 motorbikes, 1 bus, 55.1ms
image 2/20 C:\Users\krish\OneDrive\Desktop\Hackathon\src\Problem_3\test\Test_Data\data10.jpeg: 320x416 8 cars, 47.1ms
image 3/20 C:\Users\krish\OneDrive\Desktop\Hackathon\src\Problem_3\test\Test_Data\data11.jpeg: 256x416 14 cars, 3 buss, 39.7ms
image 4/20 C:\Users\krish\OneDrive\Desktop\Hackathon\src\Problem_3\test\Test_Data\data12.jpeg: 256x416 2 buss, 39.3ms
image 5/20 C:\Users\krish\OneDrive\Desktop\Hackathon\src\Problem_3\test\Test_Data\data13.jpeg: 320x416 2 cars, 5 buss, 41.4ms
image 6/20 C:\Users\krish\OneDrive\Desktop\Hackathon\src\Problem_3\test\Test_Data\data14.jpeg: 256x416 7 cars, 3 buss, 38.2ms
image 7/20 C:\Users\krish\OneDrive\Desktop\Hackathon\src\Problem_3\test\Test_Data\data15.jpeg: 256x416 1 motorbike, 35.1ms
image 8/20 C:\Users\krish\OneDrive\Desktop\Hackathon\src\Problem_3\test\Test_Data\data16.jpeg: 256x416 1 car, 1 bus, 39.5ms